In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [3]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [6]:
class X2E_Net3Smooth(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(X2E_Net3Smooth, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x=x/torch.norm(x, p=2)
        return x

In [7]:
#模型
model=X2E_Net3Smooth(512,400,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
# criterion = torch.nn.MSELoss(reduction='sum')
# criterion = torch.nn.L1Loss(reduction='sum')
criterion = torch.nn.SmoothL1Loss()
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件
se_dir='D:\\pycharm_project\\vs2ttsvoice\\SE_vector'
for i in range(80):
    print(i)
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir,pathstr)
    sepath='dpdir'+str(i+1)+'.txt' 
    sepath=os.path.join(se_dir,sepath)
    se_vectors=open(sepath).readlines()
    file=open( pathstr)
    j=0
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        se=se_vectors[j].replace('[','').replace(']','').split()
        se_filename=se[0]
        filename=data[0]
        if se_filename==filename:
#             print(se_filename)
            data=data[1:]
            se_data=se[1:]
            out=list(map(float,data))
            out=torch.tensor(out)
            pred=model(out)
            label=list(map(float,se_data))
            label=torch.tensor(label)
            loss = criterion(pred, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch+=1
            if epoch%100 == 0:
            #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
            #保存参数、
                print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
                state = { 'model':model.state_dict(), 'epoch':epoch }                       
                if not os.path.isdir('checkpointX2E'):
                     os.mkdir('checkpointX2E')    
                torch.save(state, '.\checkpointX2E\model3smooth_epoch_%d.ckpt' % (epoch))
                file_save = open('lossX2E3smooth.txt',mode='a')
                file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
                file_save.close() 
        else:continue
        j=j+1

0
epoch: 100, loss: 0.0003049
epoch: 200, loss: 0.0005026
epoch: 300, loss: 0.0001617
epoch: 400, loss: 0.0008112
epoch: 500, loss: 0.0006719
epoch: 600, loss: 0.0003576
epoch: 700, loss: 0.0003036
epoch: 800, loss: 0.000416
epoch: 900, loss: 0.0004807
epoch: 1000, loss: 0.0005557
epoch: 1100, loss: 0.0009022
epoch: 1200, loss: 0.000761
epoch: 1300, loss: 0.0008913
epoch: 1400, loss: 0.0009413
epoch: 1500, loss: 0.0006726
epoch: 1600, loss: 0.0002468
epoch: 1700, loss: 0.001392
epoch: 1800, loss: 0.0005774
1
epoch: 1900, loss: 0.0003525
epoch: 2000, loss: 0.0006035
epoch: 2100, loss: 0.0002839
epoch: 2200, loss: 0.001253
epoch: 2300, loss: 0.0002394
epoch: 2400, loss: 0.000574
epoch: 2500, loss: 0.0002816
epoch: 2600, loss: 0.0005021
epoch: 2700, loss: 0.0001841
epoch: 2800, loss: 0.001187
epoch: 2900, loss: 0.0002011
epoch: 3000, loss: 0.0004986
epoch: 3100, loss: 0.0003169
epoch: 3200, loss: 0.0005735
epoch: 3300, loss: 0.001179
epoch: 3400, loss: 0.0005953
epoch: 3500, loss: 0.00081

epoch: 27800, loss: 0.0004439
epoch: 27900, loss: 0.000478
epoch: 28000, loss: 0.0005316
epoch: 28100, loss: 0.0003914
epoch: 28200, loss: 0.000592
epoch: 28300, loss: 0.000534
epoch: 28400, loss: 0.0006745
epoch: 28500, loss: 0.0005663
epoch: 28600, loss: 0.0004188
epoch: 28700, loss: 0.0001836
epoch: 28800, loss: 0.0002409
epoch: 28900, loss: 0.0003933
epoch: 29000, loss: 0.0003855
16
epoch: 29100, loss: 0.0007778
epoch: 29200, loss: 0.0006789
epoch: 29300, loss: 0.0004424
epoch: 29400, loss: 0.0003746
epoch: 29500, loss: 0.0007468
epoch: 29600, loss: 0.0003494
epoch: 29700, loss: 0.0006072
epoch: 29800, loss: 0.0003489
epoch: 29900, loss: 0.0001395
epoch: 30000, loss: 0.0006496
epoch: 30100, loss: 0.0006028
epoch: 30200, loss: 0.0006502
epoch: 30300, loss: 0.0003153
epoch: 30400, loss: 0.0002364
epoch: 30500, loss: 0.0006649
epoch: 30600, loss: 0.000451
epoch: 30700, loss: 0.0002019
epoch: 30800, loss: 0.0004389
17
epoch: 30900, loss: 0.0003063
epoch: 31000, loss: 0.0004577
epoch: 3

epoch: 55200, loss: 0.0003753
epoch: 55300, loss: 0.00061
epoch: 55400, loss: 0.0003834
epoch: 55500, loss: 0.0003694
epoch: 55600, loss: 0.000426
epoch: 55700, loss: 0.0003781
epoch: 55800, loss: 0.0004373
epoch: 55900, loss: 0.0004776
epoch: 56000, loss: 0.0004106
epoch: 56100, loss: 0.0005802
epoch: 56200, loss: 0.0004398
epoch: 56300, loss: 0.0004943
epoch: 56400, loss: 0.0003955
epoch: 56500, loss: 0.0005238
epoch: 56600, loss: 0.0003831
epoch: 56700, loss: 0.0008384
epoch: 56800, loss: 0.0002627
epoch: 56900, loss: 0.0003531
31
epoch: 57000, loss: 0.0002743
epoch: 57100, loss: 0.0002196
epoch: 57200, loss: 0.0001374
epoch: 57300, loss: 0.0002972
epoch: 57400, loss: 0.0003584
epoch: 57500, loss: 0.0005246
epoch: 57600, loss: 0.0006563
epoch: 57700, loss: 0.0004368
epoch: 57800, loss: 0.0002566
epoch: 57900, loss: 0.0001206
epoch: 58000, loss: 0.0003018
epoch: 58100, loss: 0.0003348
epoch: 58200, loss: 0.0004494
epoch: 58300, loss: 0.0007567
epoch: 58400, loss: 0.0003803
epoch: 585

epoch: 82500, loss: 0.0001893
epoch: 82600, loss: 0.0004394
epoch: 82700, loss: 0.0003418
epoch: 82800, loss: 0.0005443
epoch: 82900, loss: 0.0005016
epoch: 83000, loss: 0.0004734
epoch: 83100, loss: 0.0005679
epoch: 83200, loss: 0.0006146
46
epoch: 83300, loss: 0.0005757
epoch: 83400, loss: 0.0002408
epoch: 83500, loss: 0.0004344
epoch: 83600, loss: 0.0006136
epoch: 83700, loss: 0.0006825
epoch: 83800, loss: 0.0003835
epoch: 83900, loss: 0.0003715
epoch: 84000, loss: 0.0002805
epoch: 84100, loss: 0.0002558
epoch: 84200, loss: 0.00017
epoch: 84300, loss: 0.0005188
epoch: 84400, loss: 0.0003275
epoch: 84500, loss: 0.0004164
epoch: 84600, loss: 0.0002354
epoch: 84700, loss: 0.0004256
epoch: 84800, loss: 0.0007702
epoch: 84900, loss: 0.0002843
epoch: 85000, loss: 0.0003773
epoch: 85100, loss: 0.0003078
47
epoch: 85200, loss: 0.0002974
epoch: 85300, loss: 0.0002948
epoch: 85400, loss: 0.0006581
epoch: 85500, loss: 0.0003187
epoch: 85600, loss: 0.0001987
epoch: 85700, loss: 0.0003105
epoch:

60
epoch: 109500, loss: 0.0004218
epoch: 109600, loss: 0.0004887
epoch: 109700, loss: 0.0007241
epoch: 109800, loss: 0.0004284
epoch: 109900, loss: 0.0003021
epoch: 110000, loss: 0.0004721
epoch: 110100, loss: 0.0003364
epoch: 110200, loss: 0.0004536
epoch: 110300, loss: 0.0003615
epoch: 110400, loss: 0.0003532
epoch: 110500, loss: 0.0005254
epoch: 110600, loss: 0.0006306
epoch: 110700, loss: 0.0002756
epoch: 110800, loss: 0.000421
epoch: 110900, loss: 0.0002388
epoch: 111000, loss: 0.0001997
epoch: 111100, loss: 0.000695
epoch: 111200, loss: 0.0002894
epoch: 111300, loss: 0.0006936
epoch: 111400, loss: 0.000448
61
epoch: 111500, loss: 0.00036
epoch: 111600, loss: 0.0002384
epoch: 111700, loss: 0.0005511
epoch: 111800, loss: 0.000532
epoch: 111900, loss: 0.0003224
epoch: 112000, loss: 0.0002964
epoch: 112100, loss: 0.0002489
epoch: 112200, loss: 0.0005718
epoch: 112300, loss: 0.0004755
epoch: 112400, loss: 0.0002459
epoch: 112500, loss: 0.00023
epoch: 112600, loss: 0.0005772
epoch: 112

epoch: 136000, loss: 0.0005356
epoch: 136100, loss: 0.0004463
epoch: 136200, loss: 0.0005996
epoch: 136300, loss: 0.0002387
epoch: 136400, loss: 0.0002585
epoch: 136500, loss: 0.0004679
epoch: 136600, loss: 0.0003184
epoch: 136700, loss: 0.0005263
epoch: 136800, loss: 0.0004183
epoch: 136900, loss: 0.0005376
epoch: 137000, loss: 0.0005795
epoch: 137100, loss: 0.0002925
epoch: 137200, loss: 0.0003521
epoch: 137300, loss: 0.0002051
74
epoch: 137400, loss: 0.0002686
epoch: 137500, loss: 0.000563
epoch: 137600, loss: 0.000461
epoch: 137700, loss: 0.0002365
epoch: 137800, loss: 0.0003544
epoch: 137900, loss: 8.644e-05
epoch: 138000, loss: 0.0003432
epoch: 138100, loss: 0.0002134
epoch: 138200, loss: 0.000206
epoch: 138300, loss: 0.0003703
epoch: 138400, loss: 0.0005135
epoch: 138500, loss: 0.0004282
epoch: 138600, loss: 0.0004782
epoch: 138700, loss: 0.0002523
epoch: 138800, loss: 0.0002347
epoch: 138900, loss: 0.0003213
epoch: 139000, loss: 0.0003822
epoch: 139100, loss: 0.0003082
epoch: 1

In [9]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\model3smooth_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [10]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the con

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5177826311842197


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5319578011877742


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


...

id10274-7rkVfW2yw0c-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.4896410244535769


id10275-CVUXDNZzcmA-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.5464136684291138


KeyboardInterrupt: 